# Woolhouse M'thly

Woolhouse's formula is a method of approximating 1/mthly life annuities from annual factors that does not depend on a fractional age assumption. It is based on the Euler-Maclaurin series expansion for the integral of a function.

## Life Annuities


$\ddot{a}^{(m)}_{x} \approx \ddot{a}_{x} - \dfrac{m-1}{2m} - \dfrac{m^2 - 1}{12m^2}(\mu_x + \delta)$

- 1/m'thly whole life annuity using the three-term Woolhouse approximation.  The third term is often omitted in practice, which leads to poor approximations in some cases.

$ \ddot{a}^{(m)}_{x:\overline{t|}} \approx \ddot{a}^{(m)}_{x} - ~_tE_x ~ \ddot{a}^{(m)}_{x+t} = \ddot{a}_{x:\overline{t|}} - \dfrac{m-1}{2m}(1 - ~_tE_x) - \dfrac{m^2 - 1}{12m^2}(\mu_x + \delta -~_tE_x(\mu_{x+t} + \delta))$

- 1/m'thly temporary life annuity from the difference of whole life Woolhouse approximations

$\overline{a}_{x} \approx \ddot{a}_{x} - \dfrac{1}{2} - \dfrac{1}{12}(\mu_x + \delta)$

- continuous life annuity with Woolhouse approximation when we let $m \to \infty$.


$\mu_x \approx -\dfrac{1}{2}(\ln p_{x-1} + \ln p_x)$

- if the force of mortality $\mu$ is not provided for the third Woolhouse term, it can be approximated from survival probabilities at integer ages.  


## Methods


Ths `Woolhouse` class implements methods to compute m'thly-pay annuity values using the Woolhouse assumption with either two or three terms.

In [1]:
from actuarialmath import Woolhouse, SULT, Recursion, UDD, Contract
import describe
describe.methods(Woolhouse)


class Woolhouse - 1/m'thly shortcuts with Woolhouse approximation

    Args:
      m : number of payments per year
      life : original fractional survival and mortality functions
      three_term : whether to include (True) or ignore (False) third term
      approximate_mu : function to approximate mu_x for third term

    Methods:
    --------

    mu_x(x, s):
      Approximates or computes mu_x for third term




## Examples


__SOA Question 7.7__

For a whole life insurance of 10,000 on (x), you are given:
- Death benefits are payable at the end of the year of death
- A premium of 30 is payable at the start of each month
- Commissions are 5% of each premium
- Expenses of 100 are payable at the start of each year
- $i = 0.05$
- $1000 A_{x+10} = 400$
- $_{10} V$ is the gross premium policy value at the end of year 10 for this insurance

Calculate $_{10} V$ using the two-term Woolhouse formula for annuities.

In [2]:
print("SOA Question 7.7:  (D) 1110")
x = 0
life = Recursion().set_interest(i=0.05).set_A(0.4, x=x+10)
a = Woolhouse(m=12, life=life).whole_life_annuity(x+10)
print(a)
contract = Contract(premium=0, benefit=10000, renewal_policy=100)
V = life.gross_future_loss(A=0.4, contract=contract.renewals())
contract = Contract(premium=30*12, renewal_premium=0.05)
V1 = life.gross_future_loss(a=a, contract=contract.renewals())
print(V, V1, V+V1)


SOA Question 7.7:  (D) 1110
 *Whole Life Annuity a_10(t=WL) <--
   a_x = (1-A_x) / d                                           ~insurance twin
      a_10(t=1) = 1                                 ~one-year discrete annuity
      a_11(t=1) = 1                                 ~one-year discrete annuity
12.141666666666666
5260.0 -4152.028174603174 1107.9718253968258


__SOA Question 6.25__:

For a fully discrete 10-year deferred whole life annuity-due of 1000 per month on (55), you are given:
- The premium, $G$, will be paid annually at the beginning of each year during the deferral period
- Expenses are expected to be 300 per year for all years, payable at the beginning of the year
- Mortality follows the Standard Ultimate Life Table
- $i = 0.05$
- Using the two-term Woolhouse approximation, the expected loss at issue is -800

Calculate $G$.

In [3]:
print("SOA Question 6.25:  (C) 12330")
life = SULT()
woolhouse = Woolhouse(m=12, life=life)
benefits = woolhouse.deferred_annuity(55, u=10, b=1000 * 12)
expenses = life.whole_life_annuity(55, b=300)
payments = life.temporary_annuity(55, t=10)
print(benefits + expenses, payments)
def fun(G):
    return life.gross_future_loss(A=benefits + expenses, a=payments,
                                  contract=Contract(premium=G))
G = life.solve(fun, target=-800, grid=[12110, 12550])
print(G)

SOA Question 6.25:  (C) 12330
98042.52569470297 8.019169307712845
12325.781125438532


__SOA Question 6.15__

For a fully discrete whole life insurance of 1000 on (x) with net premiums payable quarterly, you are given:
- $i = 0.05$
- $\ddot{a}_x = 3.4611$
- $P^{(W)}$ and $P^{(UDD)}$ are the annualized net premiums calculated using the 2-term Woolhouse (W) and the uniform distribution of deaths (UDD) assumptions,
respectively

Calculate $\dfrac{P^{(UDD)}}{P^{(W)}}$.


In [4]:
print("SOA Question 6.15:  (B) 1.002")
x = 0
life = Recursion().set_interest(i=0.05).set_a(3.4611, x=0)
A = life.insurance_twin(3.4611)
udd = UDD(m=4, life=life)
a1 = udd.whole_life_annuity(x=x)
woolhouse = Woolhouse(m=4, life=life)
a2 = woolhouse.whole_life_annuity(x=x)
print(life.gross_premium(a=a1, A=A)/life.gross_premium(a=a2, A=A))


SOA Question 6.15:  (B) 1.002
1.0022973504113772


__SOA Question 5.7__

You are given:
- $A_{35} = 0.188$
- $A_{65} = 0.498$
- $_{30}p_{35} = 0.883$
- $i = 0.04$

Calculate $1000 \ddot{a}^{(2)}_{35:\overline{30|}}$ using the two-term Woolhouse approximation.

In [5]:
print("SOA Question 5.7:  (C) 17376.7")
life = Recursion().set_interest(i=0.04)
life.set_A(0.188, x=35)
life.set_A(0.498, x=65)
life.set_p(0.883, x=35, t=30)
mthly = Woolhouse(m=2, life=life, three_term=False)
print(mthly.temporary_annuity(35, t=30))
print(1000 * mthly.temporary_annuity(35, t=30))


SOA Question 5.7:  (C) 17376.7
 *Whole Life Annuity a_35(t=WL) <--
   a_x = (1-A_x) / d                                           ~insurance twin
      a_35(t=1) = 1                                 ~one-year discrete annuity
      a_36(t=1) = 1                                 ~one-year discrete annuity
 *Whole Life Annuity a_65(t=WL) <--
   a_x = (1-A_x) / d                                           ~insurance twin
      a_65(t=1) = 1                                 ~one-year discrete annuity
      a_66(t=1) = 1                                 ~one-year discrete annuity
 *Pure Endowment E_35(t=30) <--
   E_35(t=30) = p_35(t=30) * v(t=30)                           ~pure endowment
17.37671459632958
 *Whole Life Annuity a_35(t=WL) <--
   a_x = (1-A_x) / d                                           ~insurance twin
      a_35(t=1) = 1                                 ~one-year discrete annuity
      a_36(t=1) = 1                                 ~one-year discrete annuity
 *Whole Life Annuity 